# Semantic Scholar API
100 requests per 5 minutes
The API allows up to 100 requests per 5 minutes. 

To access a higher rate limit, complete the form to request authentication for your project.

Max limit for each request is 100, so every 5 min we can gather (100*100)* amount of people

Issue/Drawback of semantic scholar: it needs a search term, meaning, the topics/industry/niche needs to be chosen 

In [4]:
import requests
import json
import pandas as pd
import time
import string
import json
import os
from dotenv import load_dotenv
load_dotenv()


True

In [5]:
# Parameters for retrieving data
amount_of_data = 100  # Total amount of data to retrieve
start_date = 2019  # Year of publication
end_date = 2023  # Year of publication
journal = ["Nat. Chem. Biol"]
# research_field = ['Computer Science'] # Research field(s) to query (as a list)
alphabet_letters = ['A','E','I','U','O'] #  list(string.ascii_uppercase) # Research field(s) to query (as a list)
initial_offset = 0  # Initial start offset
number_of_publications_per_request = 100

# Gentle fetching
sleep_duration = 1  # Sleep duration in seconds between requests
retry_sleep_duration = 1  # Sleep duration in seconds between requests
max_retries = 100  # Maximum number of retries for failed requests

# Define the API endpoint
api_url = 'https://api.semanticscholar.org/graph/v1/paper/search'
length_df = 0

In [50]:
# Retrieving single publication for debugging purposes
params = {
    'query': "A",  # Use the first research field from the list
    'venue': journal[0],
    'year': 2013,
    'limit': 1,  # Number of results per request
    'offset': 0,  # Offset for pagination
    # 'fields': 'externalIds,title,abstract,citationCount,year',
    'fields': 'externalIds,title,authors,abstract,citationCount,referenceCount,influentialCitationCount,fieldsOfStudy,s2FieldsOfStudy,publicationTypes,publicationDate,publicationVenue,year,tldr,embedding.specter_v2,citations,references',
}

# Make the GET request
response = requests.get(api_url, params=params,headers={'X-API-KEY':os.getenv("SEMANTICSCHOLAR_API_KEY")})
# Check if the request was successful
if response.status_code == 200:
    data = json.loads(response.text)
else:
    print(f"Error: Request failed with status code {response.status_code}")
    print(f"Error message: {response.text}")


In [60]:
def flatten_dict(d, parent_key='', separator='_'):
    flattened = {}
    for key, value in d.items():
        new_key = parent_key + separator + key if parent_key else key
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                sub_new_key = new_key + separator + sub_key
                flattened[sub_new_key] = sub_value
        else:
            flattened[new_key] = value
    return flattened

flattened_data = []  # A list to store flattened dictionaries
original_data = pd.DataFrame(data)  # Original DataFrame
num_records = len(data['data'])  # Number of records in the 'data' list

# Loop through each record in 'data' and flatten it
for i in range(num_records):
    flattened_data.append(flatten_dict(data['data'][i]))

# Create a DataFrame from the flattened data
flattened_df = pd.DataFrame(flattened_data)
flattened_df.to_csv("./temp_data/all_features.csv",index=False)
# Display the first few rows of the flattened DataFrame
display(flattened_df.head())

,paperId,externalIds_MAG,externalIds_DOI,externalIds_CorpusId,externalIds_PubMed,publicationVenue_id,publicationVenue_name,publicationVenue_type,publicationVenue_alternate_names,publicationVenue_issn,...,s2FieldsOfStudy,tldr_model,tldr_text,publicationTypes,publicationDate,embedding_model,embedding_vector,authors,citations,references
0,08e50d1cc59a1d9c1a0d21dfe9667e243aa1094d,1984615306,10.1016/S0140-6736(13)61611-6,41940846,23993280,ba627f0e-abf9-4f01-8146-73118f4ebaf9,The Lancet,journal,[Lancet],0923-7577,...,"[{'category': 'Medicine', 'source': 'external'...",tldr@v2.0.0,The findings show the striking and growing cha...,"[Review, JournalArticle]",2013-11-09,specter_v2,"[0.24848510324954987, 0.40732088685035706, -0....","[{'authorId': '5260255', 'name': 'H. Whiteford...",[{'paperId': '53840ae196ecf4d46c5f02ed535beafe...,[{'paperId': '00f63e8997831bbe70cd2bb949c44dbd...


In [27]:
# Fetching Data
year_list = list(range(start_date, end_date + 1))
retries_retry_exhausted = False
year_list.reverse()
for j, year in enumerate(year_list):
    print("Year",year)
    for k,letter in enumerate(alphabet_letters):
        # First request to get the number of total papers to limit the loop
        params = {
            'query': alphabet_letters[k],  # Use the first research field from the list
            'venue': journal[0],
            'year': year,
            'limit': 1,  # Number of results per request
            'offset': 0,  # Offset for pagination
            'fields': 'title,authors,abstract,citationCount,year',
            # 'fields': 'externalIds,title,authors,abstract,citationCount,referenceCount,influentialCitationCount,fieldsOfStudy,s2FieldsOfStudy,publicationTypes,publicationDate,citations,references,publicationVenue,year',

        }

        # Make the GET request
        response = requests.get(api_url, params=params,headers={'x-api-key':os.getenv("SEMANTICSCHOLAR_API_KEY")})
        # response = requests.get(api_url, params=params)
        time.sleep(1)

        # Check if the request was successful
        if response.status_code == 200:
            data = json.loads(response.text)

        # calculate the batches required to get every publication for that topic
        num_batches, remainder = divmod(data['total'], number_of_publications_per_request)
        batches = [number_of_publications_per_request] * num_batches
        if remainder > 0:
            batches.append(remainder)
            
        print("For the letter: ", letter ," batches: ",batches)

        for i,batch in enumerate(batches):
            offset = initial_offset + number_of_publications_per_request * i  # Calculate the current offset
            retries = 0

            print("Letter: ", alphabet_letters[k]," Batch: ",str(batches[i]), " Iteration: ", i)
            while retries < max_retries:
                # Define query parameters
                params = {
                    'query': alphabet_letters[k],  # Use the first research field from the list
                    'venue': journal[0],
                    'year': year,
                    'limit': batch,  # Number of results per request
                    'offset': offset,  # Offset for pagination
                    # 'fields': 'externalIds,title,authors,abstract,citationCount,referenceCount,influentialCitationCount,fieldsOfStudy,s2FieldsOfStudy,publicationTypes,publicationDate,citations,references,publicationVenue,year',
                    'fields': 'externalIds,title,abstract,citationCount,referenceCount,influentialCitationCount,fieldsOfStudy,s2FieldsOfStudy,publicationTypes,publicationDate,publicationVenue,year',
                }

                # Make the GET request
                response = requests.get(api_url, params=params,headers={'x-api-key':os.getenv("SEMANTICSCHOLAR_API_KEY")})
                # response = requests.get(api_url, params=params)
                # Check if the request was successful
                if response.status_code == 200:
                    data = json.loads(response.text)

                    # Create a DataFrame for unfiltered results
                    df = pd.DataFrame(data['data'])

                    break  # Successful request, exit the retry loop
                else:
                    print(f"Error (Attempt {retries + 1}):", response.status_code)
                    retries += 1
                    if retries < max_retries:
                        print("Retrying after sleep: "+str(retry_sleep_duration*1)+"min")
                        time.sleep(retry_sleep_duration*1)
                    if retries == max_retries:
                        print("Fetch Failed")
                        retries_retry_exhausted = True
                        break
                if retries_retry_exhausted == True:
                    print("Fetch Failed")
                    break
            if retries_retry_exhausted == True:
                print("Fetch Failed")
                break
            # print("total amount",response.json())
            length_df += len(df)  # Update the total length of unfiltered data

            # Save each iteration into a separate file - to keep some of the data in case of an error
            # The filename format includes the research field, offset, and is converted to lowercase
            df.to_csv('./data/'+ str(year) +"_"+ journal[0]+ "_" + alphabet_letters[k].replace(" ", "_").lower() + "_" + str(offset) + '.csv', index=False)

            # Sleep for a specified duration between requests
            time.sleep(sleep_duration)
    if retries_retry_exhausted == True:
        print("Fetch Failed")
        break
    # Create an empty DataFrame to store the merged data
    merged_data = pd.DataFrame()
    # Iterate through CSV files in the directory
    for filename in os.listdir("./data/"):
        if filename.endswith(".csv"):
            # Extract the year from the filename (assuming the filename follows the "year_" format)
            file_year = int(filename.split("_")[0])

            if file_year == year:
                # Read the CSV file into a DataFrame
                temp_df = pd.read_csv("./data/"+ filename)
                merged_data = pd.concat([merged_data, temp_df], ignore_index=True)
                # Remove the old CSV file
                os.remove("./data/"+ filename)
    # Calculate the number of rows dropped
    dropped_count = len(merged_data) - len(merged_data.drop_duplicates(subset='paperId', keep='first'))

    # Remove rows with duplicate 'externalIdsIDs' values
    merged_data = merged_data.drop_duplicates(subset='paperId', keep='first')

    # Calculate the number of rows remaining
    remaining_count = len(merged_data)

    # Print the counts
    print(f"Removed {dropped_count} rows. Rows remaining: {remaining_count}")
    merged_data.to_csv('./data/'+ str(year) +"_"+ journal[0]+"_"+str(remaining_count)+'no_cit_no_ref_no_auth.csv', index=False)
    time.sleep(sleep_duration)
 


# Print the total length of unfiltered data after all iterations
print("Length of the data", length_df)


Year 2023


For the letter:  I  batches:  [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,

KeyboardInterrupt: 

In [ ]:
merged_data.head().T

In [ ]:
df.head().T

# Single Query - Using the Batch Endpoint

In [ ]:
api_url = 'https://api.semanticscholar.org/graph/v1/paper/search'
# https://api.semanticscholar.org/graph/v1/paper/search?query=computer+science&year=2015&fields=title,year,authors,citationCount&limit=50


# Rest of the code is not required but it provides example how to filter the data before saving

In [ ]:
# # Fetching and filtering at the same time if required
# # Fetching Data
# length_filtered_df = 0
# length_unfiltered_df = 0

# for i in range(int(amount_of_data/100)):
#     offset = initial_offset + 100 * i
#     retries = 0
    
#     while retries < max_retries:
#         # Define query parameters
#         params = {
#             'query': research_field,
#             'year': year_of_publication,
#             'limit': 100,
#             'offset': offset,
#             'fields': 'title,authors,abstract,citationCount,year',
#         }

#         # Make the GET request
#         response = requests.get(api_url, params=params)

#         # Check if the request was successful
#         if response.status_code == 200:
#             data = json.loads(response.text)

#             # Create a DataFrame for unfiltered results
#             df_unfiltered = pd.DataFrame(data['data'])

#             # Filter the results based on 'citationCount' > 10
#             filtered_results = [paper for paper in data['data'] if paper.get('citationCount', 0) > 10]

#             # Create a DataFrame for filtered results
#             df_filtered = pd.DataFrame(filtered_results)


#             break  # Successful request, exit the retry loop
#         else:
#             print(f"Error (Attempt {retries + 1}):", response.status_code)
#             retries += 1
#             if retries < max_retries:
#                 print("Retrying after sleep...")
#                 time.sleep(sleep_duration)
    
#     # length_filtered_df = len(df_filtered)
#     length_unfiltered_df += len(df_unfiltered)

#     # Save each iteration into separate file - to keep some of the data in case of an error
#     df_unfiltered.to_csv('./data/'+research_field.replace(" ","_").lower() + "_" + str(offset) + '.csv', index=False)
#     df_filtered.to_csv('./data/df_filtered' + str(offset) + '.csv', index=False)

#     # Sleep for a specified duration between requests
#     print("Research field",research_field,"Iteration: ", i)
#     time.sleep(sleep_duration)
# print("Length of the filtered data",length_unfiltered_df)
# print("Length of the unfiltered data",length_filtered_df)